# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataManipulationHW").getOrCreate()
spark

24/01/27 17:36:54 WARN Utils: Your hostname, Saurabh-Guptas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.116 instead (on interface en0)
24/01/27 17:36:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/27 17:36:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [95]:
path = '/Users/apple/Desktop/Learning_PySpark/Manipulating_Dataframes/'

data = spark.read.csv(path+'Rep_vs_Dem_tweets.csv',inferSchema=True,header=True)

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [71]:
data.limit(5).show()

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         NULL|                NULL|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+



**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [72]:
data.limit(5).show(truncate=False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [73]:
data.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [74]:
from pyspark.sql.functions import *
data.select(data.Tweet,(regexp_extract(data.Tweet,'@LatinoLeader',0)).alias('Regex_Found')).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Tweet                                                                                                                                       |Regex_Found  |
+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |             |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|             |
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |             |
|NULL                                                     

## 2. Replace any value other than 'Democrat' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrat' or 'Republican' in the Party column. We are assuming that this is dirty data that needs to be cleaned up.

In [81]:
parties = data.selectExpr("Party","Case when Party in ('Democrat','Republican') then Party else 'Other' end as Party_new")
parties.show(truncate=False)

+----------------------------------------------------+---------+
|Party                                               |Party_new|
+----------------------------------------------------+---------+
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Congress has allocated about $18…"                  |Other    |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                                            |Democrat |
|Democrat                

In [84]:
parties.groupBy("Party_new").count().orderBy(desc("count")).show(truncate=False)


+----------+-----+
|Party_new |count|
+----------+-----+
|Republican|44392|
|Democrat  |42068|
|Other     |6029 |
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [10]:
data.withColumn("Tweet", regexp_replace(data.Tweet, "https://[^\s]+",'')).show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

## 4. Remove any leading or trailing white space in the tweet column

In [11]:
data.withColumn('Tweet', trim(data.Tweet)).show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [12]:
data = data.withColumnRenamed('Party', 'Dem_Rep')
data.limit(4).toPandas()

,Dem_Rep,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [13]:
data.select(concat_ws(' ', data.Dem_Rep, data.Handle).alias("concat_col_name")).show(5,truncate=False)

+----------------------------------+
|concat_col_name                   |
+----------------------------------+
|Democrat RepDarrenSoto            |
|Democrat RepDarrenSoto            |
|Democrat RepDarrenSoto            |
|Congress has allocated about $18…"|
|Democrat RepDarrenSoto            |
+----------------------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [171]:
from pyspark.sql.functions import *
# Parenthesis are used to mark a subexpression within a larger expression
# The . matches any character other than a new line
# | means is like or
# \w+ means followed by any word
pattern = '(.|'')(#)(\w+)'
# * is used to match the preceding character zero or more times.
# ? will match the preceding character zero or one times, but no more.
# $ is used to match the ending position in a string. 
split_pattern = r'.*?({pattern})'.format(pattern=pattern)
end_pattern = r'(.*{pattern}).*?$'.format(pattern=pattern)

# $1 here means to capture the first part of the regex result
# The , will separate each find with a comma in the a array we create
df2 = data.withColumn('a', regexp_replace('Tweet', split_pattern, '$1,')).where(col('Tweet').like('%#%'))
df2.select('a').show(3,False)
# Remove all the other results that came up
df3 = df2.withColumn('a', regexp_replace('a', end_pattern, '$1'))
df3.select('a').show(3,False)
# Finally create an array from the result by splitting on the comma
df4 = df3.withColumn('a', split('a', r','))
df4.select('a').show(3,False)
df4.limit(3).toPandas()

+-----------------------------------------------------------------------------------------+
|a                                                                                        |
+-----------------------------------------------------------------------------------------+
| #SaveTheInternet, #NetNeutrality, legislation here in the House… https://t.co/n3tggDLU1L|
| #NALCABPolicy2018,.…                                                                    |
| #NetNeutrality, rules. Find out…                                                        |
+-----------------------------------------------------------------------------------------+
only showing top 3 rows

+---------------------------------+
|a                                |
+---------------------------------+
| #SaveTheInternet, #NetNeutrality|
| #NALCABPolicy2018               |
| #NetNeutrality                  |
+---------------------------------+
only showing top 3 rows

+------------------------------------+
|a             

,Party,Handle,Tweet,a
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...","[ #SaveTheInternet, #NetNeutrality]"
1,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[ #NALCABPolicy2018]
2,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...,[ #NetNeutrality]


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [98]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [99]:
df = df.withColumn("dob", df['dob'].cast(DateType()))\
        .withColumn("visit1", df['visit1'].cast(DateType()))\
        .withColumn("visit2", df['visit2'].cast(DateType()))\
        .withColumn("visit3", df['visit3'].cast(DateType()))

df.show(truncate=False)
print(df.printSchema())


+----------+---------+----------+----------+----------+----------+
|first_name|last_name|dob       |visit1    |visit2    |visit3    |
+----------+---------+----------+----------+----------+----------+
|Mohammed  |Alfasy   |1987-04-08|2016-01-07|2017-02-03|2018-03-02|
|Marcy     |Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|
|Ginny     |Ginger   |1986-07-10|2014-08-07|2015-02-03|2016-03-02|
|Vijay     |Doberson |1988-05-02|2016-01-07|2018-02-03|2018-03-02|
|Orhan     |Gelicek  |1987-05-11|2016-05-07|2017-01-03|2018-09-02|
|Sarah     |Jones    |1956-07-06|2016-04-07|2017-08-03|2018-10-02|
|John      |Johnson  |2017-10-12|2018-01-02|2018-10-03|2018-03-02|
+----------+---------+----------+----------+----------+----------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)

None


## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [121]:
diff_visits = df.select(df.visit1, df.visit2, df.visit3, (date_diff(df.visit2,df.visit1)).alias('diff1'),(date_diff(df.visit3,df.visit2)).alias('diff2'))
diff_visits = diff_visits.withColumn("total_visits",diff_visits['diff1']+diff_visits['diff2'])
diff_visits.show(truncate=False)
diff_visits.select(mean(diff_visits.total_visits).alias("avg_visits")).show()

+----------+----------+----------+-----+-----+------------+
|visit1    |visit2    |visit3    |diff1|diff2|total_visits|
+----------+----------+----------+-----+-----+------------+
|2016-01-07|2017-02-03|2018-03-02|393  |392  |785         |
|2015-01-07|2017-01-03|2018-01-02|727  |364  |1091        |
|2014-08-07|2015-02-03|2016-03-02|180  |393  |573         |
|2016-01-07|2018-02-03|2018-03-02|758  |27   |785         |
|2016-05-07|2017-01-03|2018-09-02|241  |607  |848         |
|2016-04-07|2017-08-03|2018-10-02|483  |425  |908         |
|2018-01-02|2018-10-03|2018-03-02|274  |-215 |59          |
+----------+----------+----------+-----+-----+------------+

+-----------------+
|       avg_visits|
+-----------------+
|721.2857142857143|
+-----------------+



## 8. Can you calculate the age of each patient?

In [123]:
df.select(df.dob, (date_diff(current_date(),df.dob)/365).alias('age')).show(truncate=False)

+----------+-----------------+
|dob       |age              |
+----------+-----------------+
|1987-04-08|36.83013698630137|
|1986-04-08|37.83013698630137|
|1986-07-10|37.57534246575342|
|1988-05-02|35.76164383561644|
|1987-05-11|36.73972602739726|
|1956-07-06|67.6054794520548 |
|2017-10-12|6.295890410958904|
+----------+-----------------+



## 9. Can you extract the month from the first visit column and call it "Month"?

In [105]:
df.selectExpr("visit1", "MONTH(visit1) as Month").show(truncate=False)

+----------+-----+
|visit1    |Month|
+----------+-----+
|2016-01-07|1    |
|2015-01-07|1    |
|2014-08-07|8    |
|2016-01-07|1    |
|2016-05-07|5    |
|2016-04-07|4    |
|2018-01-02|1    |
+----------+-----+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [49]:
supermarket_df = spark.read.csv(path+'supermarket_sales.csv', inferSchema=True,header=True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [50]:
supermarket_df.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,2024-01-27 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,2024-01-27 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,2024-01-27 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,2024-01-27 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,2024-01-27 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


In [51]:
supermarket_df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [52]:
supermarket_df = supermarket_df.withColumn("Date", to_date(supermarket_df.Date, "M/d/yyyy"))
supermarket_df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



In [53]:
supermarket_df.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,2024-01-27 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,2024-01-27 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,2024-01-27 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,2024-01-27 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,2024-01-27 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [57]:
supermarket_df.selectExpr("Date", "Month(Date) as months").show(truncate=False)

+----------+------+
|Date      |months|
+----------+------+
|2019-01-05|1     |
|2019-03-08|3     |
|2019-03-03|3     |
|2019-01-27|1     |
|2019-02-08|2     |
|2019-03-25|3     |
|2019-02-25|2     |
|2019-02-24|2     |
|2019-01-10|1     |
|2019-02-20|2     |
|2019-02-06|2     |
|2019-03-09|3     |
|2019-02-12|2     |
|2019-02-07|2     |
|2019-03-29|3     |
|2019-01-15|1     |
|2019-03-11|3     |
|2019-01-01|1     |
|2019-01-21|1     |
|2019-03-11|3     |
+----------+------+
only showing top 20 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [157]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the review_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [158]:
reviews = reviews.withColumn("review_txt_cleaned", regexp_replace(reviews.review_txt, '[^\sa-zA-Z0-9]',""))
reviews = reviews.withColumn("review_txt_cleaned", lower(reviews.review_txt_cleaned))
reviews = reviews.withColumn("review_txt_cleaned", trim(reviews.review_txt_cleaned))
reviews = reviews.withColumn("review_txt_array", split(reviews.review_txt_cleaned," "))
reviews.show(truncate=False)

+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |review_txt_cleaned                                                                 |review_txt_array                                                                                   |
+------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |epic this is the best movie i have ever seen                     

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [160]:
epic = reviews.withColumn("result", array_contains(col("review_txt_array"), "epic"))
epic.toPandas()

,rating,review_txt,review_txt_cleaned,review_txt_array,result
0,5,Epic. This is the best movie I have EVER seen,epic this is the best movie i have ever seen,"[epic, this, is, the, best, movie, i, have, ev...",True
1,4,"Pretty good, but I would have liked to seen be...",pretty good but i would have liked to seen bet...,"[pretty, good, but, i, would, have, liked, to,...",False
2,3,So so. Casting could have been improved,so so casting could have been improved,"[so, so, casting, could, have, been, improved]",False
3,5,The most EPIC movie of the year! Casting was a...,the most epic movie of the year casting was aw...,"[the, most, epic, movie, of, the, year, castin...",True
4,4,Solid but I would have liked to see more of th...,solid but i would have liked to see more of th...,"[solid, but, i, would, have, liked, to, see, m...",False
5,5,THE BOMB!!!!!!!,the bomb,"[the, bomb]",False


### That's it! Great Job!